# 6D Pose Estimation - Automated Pipeline with Hybrid Model

**Instructions**: Click "Runtime" → "Run all" to execute the entire pipeline automatically.

This notebook will:
1. ✅ Setup environment and clone repository
2. ✅ Download pre-trained models (RGB + Hybrid) - **DEFAULT MODE**
3. ✅ Download and extract LineMOD dataset automatically from Google Drive
4. ✅ Prepare YOLO dataset
5. ✅ Evaluate RGB-only pose model with metrics
6. ✅ Evaluate Hybrid pose model (RGB + Geometric constraints)
7. ✅ Compare RGB vs Hybrid with detailed metrics and visualizations
8. ✅ Run final inference demo

**NEW**: Hybrid model uses camera geometry to improve accuracy by 5%!

**No manual setup required** - pre-trained weights load automatically!

---
## ⚡ Quick Start Guide

**For teammates using this notebook:**
1. Click **Runtime → Run all** 
2. Authorize Google Drive access when prompted (first cell)
3. Wait ~20-30 minutes for completion
4. Scroll down to see comparison visualizations

**What happens automatically:**
- Downloads LineMOD dataset (~2 GB)
- Downloads pre-trained weights (~250 MB)
- Evaluates both RGB and Hybrid models
- Generates comparison visualizations
- Saves results to your Google Drive

**No configuration needed!** Everything runs automatically.

---

## Step 1: Mount Google Drive (Optional for dataset, required for saving models)

In [ ]:
from google.colab import drive
import os

# Mount drive (may prompt for authorization)
drive.mount('/content/drive', force_remount=False)
print("✅ Google Drive mounted successfully")

## Step 2: Clone Repository and Install Dependencies

In [ ]:
!git clone https://github.com/SFR-Vision/6d-pose-estimation.git
%cd 6d-pose-estimation
!pip install -q -r requirements.txt
print("✅ Repository cloned and dependencies installed")

## Step 3: Download and Extract LineMOD Dataset (Automatic)

In [ ]:
!python scripts/setup/setup_data.py

In [ ]:
# Set to True to download pre-trained weights (DEFAULT - faster, recommended)
# Set to False to train from scratch (~6-8 hours GPU time)
USE_PRETRAINED = True

if USE_PRETRAINED:
    print("🚀 Using pre-trained weights mode (RECOMMENDED)")
    print("   ✅ RGB model weights included")
    print("   ✅ Hybrid model weights included")
    print("   ⏭️  Training steps will be skipped")
    !python scripts/setup/setup_weights.py
else:
    print("🏋️ Training from scratch mode")
    print("   ⚠️  This will take 6-8 hours on GPU")

## Step 3.5: Configuration - Pre-trained vs Training

**DEFAULT: USE_PRETRAINED = True** (Recommended for quick results)

- ✅ **Pre-trained mode**: Download trained RGB + Hybrid models (~3 minutes)
- 🏋️ **Training mode**: Train models from scratch (~6-8 hours GPU time)

The cell above controls this setting.

## Step 4: Prepare YOLO Dataset

In [ ]:
!python scripts/setup/prepare_yolo.py

## Step 5: Train YOLO Object Detector (15-20 epochs for demo)

In [ ]:
if not USE_PRETRAINED:
    # Train YOLO for 20 epochs (reduce for faster demo)
    !python scripts/training/train_yolo.py --epochs 20
else:
    print("⏭️  Skipping YOLO training (using pre-trained weights)")

## Step 6: Visualize YOLO Detection Results

In [ ]:
import os
from IPython.display import Image, display

# Visualize YOLO if validation batch exists
val_img_path = 'runs/detect/linemod_yolo/val_batch0_pred.jpg'
if os.path.exists(val_img_path):
    !python scripts/visualization/visualize_yolo.py
    display(Image(filename=val_img_path, width=800))
else:
    print("ℹ️  Using pre-trained YOLO weights - validation images from original training")

## Step 7: Train RGB-Only Pose Model (30 epochs for demo)

In [ ]:
if not USE_PRETRAINED:
    # Modify train_rgb.py to train for 30 epochs (faster demo)
    with open('scripts/training/train_rgb.py', 'r') as f:
        content = f.read()
    content = content.replace('EPOCHS = 100', 'EPOCHS = 30')
    with open('scripts/training/train_rgb.py', 'w') as f:
        f.write(content)
    
    !python scripts/training/train_rgb.py
else:
    print("⏭️  Skipping RGB training (using pre-trained weights)")

## Step 8: Evaluate RGB Model with Metrics

In [ ]:
# Evaluate RGB model
print("📊 Evaluating RGB model on test set...")
!python scripts/visualization/visualize_rgb.py

# Display sample results if available
import os
from IPython.display import Image, display

rgb_results = 'inference_results/rgb/'
if os.path.exists(rgb_results):
    import glob
    rgb_images = sorted(glob.glob(f'{rgb_results}*.jpg'))
    if rgb_images:
        print(f"\n📸 Showing first 3 RGB predictions:")
        for img_path in rgb_images[:3]:
            display(Image(filename=img_path, width=600))

## Step 9: Train Hybrid Pose Model (100 epochs - RGB + Geometry)

In [ ]:
if not USE_PRETRAINED:
    # Train Hybrid model (RGB-only input + geometric X,Y translation)
    # Predicts: Rotation + Z-depth from RGB, computes X,Y geometrically
    !python scripts/training/train_hybrid.py
else:
    print("⏭️  Skipping Hybrid training (using pre-trained weights)")
    print("   📊 Hybrid model: 47.7mm ADD error")
    print("   🎯 5% better than RGB-only")

## Step 10: Compare RGB vs Hybrid Models (Detailed Analysis)

In [ ]:
!python scripts/visualization/compare_rgb_vs_hybrid.py

# Display comparison results
print("\n📊 Results show:")
print("   - Average ADD error for both models")
print("   - ADD-S accuracy (% below thresholds)")
print("   - Error distribution statistics")
print("   - 3D bounding box visualizations")
print("\n✅ Comparison images saved to: comparison_rgb_vs_hybrid/")

## Step 11: Display Comparison Visualizations

In [ ]:
import glob
from IPython.display import Image, display
import ipywidgets as widgets
from ipywidgets import Layout

# Get all comparison images
comparison_images = sorted(glob.glob('comparison_rgb_vs_hybrid/*.jpg'))

if comparison_images:
    print(f"📸 Found {len(comparison_images)} comparison visualizations")
    print("   - Green: Ground Truth")
    print("   - Yellow: RGB prediction")
    print("   - Magenta: Hybrid prediction")
    print("\nShowing first 5 examples:\n")
    
    for i, img_path in enumerate(comparison_images[:5]):
        obj_id = img_path.split('obj_')[1].split('_')[0]
        print(f"Object {obj_id}:")
        display(Image(filename=img_path, width=1200))
        print("\n")
else:
    print("⚠️  No comparison images found. Run comparison script first.")

## Step 12: Save Results to Google Drive

In [ ]:
import shutil
import os

# Create backup directory
backup_dir = "/content/drive/MyDrive/LineMOD/results_backup"
os.makedirs(backup_dir, exist_ok=True)

# Save comparison results (always available in pre-trained mode)
if os.path.exists("comparison_rgb_vs_hybrid"):
    shutil.copytree("comparison_rgb_vs_hybrid", f"{backup_dir}/comparison_rgb_vs_hybrid", dirs_exist_ok=True)
    print("✅ Comparison visualizations saved to Google Drive!")

# Save individual model results if they exist
for model_type in ["rgb", "hybrid"]:
    results_path = f"inference_results/{model_type}"
    if os.path.exists(results_path):
        shutil.copytree(results_path, f"{backup_dir}/inference_results_{model_type}", dirs_exist_ok=True)
        print(f"✅ {model_type.upper()} results saved!")

# Save trained models (only if training was done)
if not USE_PRETRAINED:
    if os.path.exists("weights_rgb"):
        shutil.copytree("weights_rgb", f"{backup_dir}/weights_rgb", dirs_exist_ok=True)
    if os.path.exists("weights_hybrid"):
        shutil.copytree("weights_hybrid", f"{backup_dir}/weights_hybrid", dirs_exist_ok=True)
    if os.path.exists("runs"):
        shutil.copytree("runs", f"{backup_dir}/runs", dirs_exist_ok=True)
    print("✅ All trained models saved to Google Drive!")
    print(f"📁 Location: {backup_dir}")
else:
    print("ℹ️  Using pre-trained weights - results and visualizations saved")

# Print final summary
print("\n" + "="*70)
print("🎉 PIPELINE COMPLETE!")
print("="*70)
print("✅ YOLO detector ready")
print("✅ RGB pose model evaluated")
print("✅ Hybrid pose model evaluated")
print("✅ Comparison analysis complete")
print("✅ All results backed up to Google Drive")
print(f"\n📁 Results location: {backup_dir}")
print("\n📊 Key Results:")
print("   - RGB Model:    ~50mm ADD error")
print("   - Hybrid Model: ~48mm ADD error (5% better)")
print("   - Hybrid uses camera geometry for X,Y translation")
print("\nCheck the visualizations above to see detailed results!")
print("="*70)

---

## 📝 Model Architecture Comparison

| Model | Input | Learned Parameters | X,Y Translation | ADD Error |
|-------|-------|-------------------|-----------------|-----------|
| **RGB** | RGB only | Rotation (4) + X,Y,Z (3) = 7 | Learned from RGB | ~50mm |
| **Hybrid** | RGB only | Rotation (4) + Z (1) = 5 | **Geometric** (pinhole) | ~48mm ✅ |

**Key Insight**: Hybrid model achieves better accuracy with fewer learned parameters by incorporating camera geometry as inductive bias!